# Régression

## Objectif

Dans cette partie "Régression", on cherche à prédire les émissions de CO2 de voitures, en se basant sur leurs caractéristiques \(poids, carburant utilisé, etc\). L'objectif est de minimiser la moyenne des erreurs absolues entre les valeurs prédites et les valeurs réelles.

## Importation

Nos données d'entraînement sont situées dans des fichiers CSV. Dans un premier temps, on cherche donc à importer ces données CSV.



In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [3]:
df = pd.read_csv('dataset/train.csv')
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']

X = df.loc[:, ~df.columns.isin(['id', 'co2'])].select_dtypes(include=numerics).to_numpy()
y = df['co2'].to_numpy()

print(X[0])
print(y[0])

[7.00000000e+01 1.97600000e+03 2.07500000e+03 9.10000038e+00
 6.40000010e+00 7.40000010e+00 8.29999970e-02            nan
 2.29000002e-01 2.50000000e-01 1.00000000e-03]
195


## Premier modèle

D'abord on construit un premier modèle dit "pyramidal". L'idée est de partir avec un nombre d'entrées fixé, correspondant aux caractéristiques de nos voitures. Dans notre cas, on commence pour l'instant avec les 11 paramètres numériques présents dans nos données \(en enlevant les paramètres "id" et "co2"\). On place ensuite une première couche cachée avec un grand nombre de neurones, avant de faire décroître ce nombre sur les couches suivantes.

On y ajoute aussi la fonction d'activation ReLU (Rectified Linear Unit). Elle calcule simplement le maximum entre 0 et le niveau d'activation du neurone.



In [4]:
# Define the model
model = nn.Sequential(
    nn.Linear(11, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Premier entraînement

Une fois le réseau de neurone défini, on va entraîner ce modèle. Pour l'entraînement, on définit une fonction de perte qui nous permettra de quantifier l'écart entre les prévisions et les observations réelles. On utilise pour cela la fonction MSE (Mean Square Error) : $$ MSE = \frac{1}{n} \sum_{i=1}^{n} (\hat{Y_i} - Y_i)^2$$

Pour optimiser le processus, on utilise l'algorithme Adam. Cet algorithme implémente une méthode stochastique très efficace de descente de gradient pour modifier les poids des neurones à chaque itération de l'entraînement.



In [5]:
# train-test split of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

In [6]:
n_epochs = 100   # number of epochs to run
batch_size = 10  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

In [7]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []

In [8]:
# Entrainement
 
for epoch in range(n_epochs):
    model.train()
    with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(mse=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())

KeyboardInterrupt: 

In [ ]:
# restore model and return best accuracy
model.load_state_dict(best_weights)
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()